<a href="https://colab.research.google.com/github/Rogerio5/Yolo-Network-Detection-Project/blob/main/Projeto_YOLO_Visao_Computacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📦 Instalar dependências
!apt-get update && apt-get install -y python3-pip unzip
!pip install pandas matplotlib mean_average_precision plotly opencv-python

# 📥 Clonar OIDv4 Toolkit e baixar imagens rotuladas
!git clone https://github.com/EscVM/OIDv4_ToolKit.git
%cd OIDv4_ToolKit
!mkdir -p OID/Dataset
!python3 main.py downloader --classes Helmet Safety_vest --type_csv train --limit 100
!python3 convert_annotations.py --dataset_version "OIDv4" --model yolov3

# ⚙️ Clonar Darknet e compilar
%cd ..
!git clone https://github.com/pjreddie/darknet
%cd darknet

USE_GPU = True
MODEL_TINY = False

if USE_GPU:
    !sed -i 's/GPU=0/GPU=1/' Makefile
    !sed -i 's/CUDNN=0/CUDNN=1/' Makefile
    !sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!make -j8

# 📥 Baixar pesos pré-treinados
!wget https://pjreddie.com/media/files/darknet53.conv.74

# 🗂️ Preparar arquivos de configuração
!mkdir -p data/obj backup

classes = ["capacete", "colete"]
with open("data/obj.names", "w") as f:
    f.write("\n".join(classes))
with open("data/obj.data", "w") as f:
    f.write(f"""classes= {len(classes)}
train  = data/train.txt
valid  = data/valid.txt
names  = data/obj.names
backup = backup/
""")

# 📝 Gerar listas de treino e validação
import os, random

img_dir = "../OIDv4_ToolKit/OID/Dataset/train"
img_files = [os.path.join(root, f) for root, _, files in os.walk(img_dir) for f in files if f.endswith(".jpg")]
random.shuffle(img_files)
split = int(0.8 * len(img_files))
train_imgs, valid_imgs = img_files[:split], img_files[split:]

with open("data/train.txt", "w") as f:
    f.write("\n".join(train_imgs))
with open("data/valid.txt", "w") as f:
    f.write("\n".join(valid_imgs))

# ⚙️ Configurar yolov3.cfg
cfg_base = "yolov3.cfg" if not MODEL_TINY else "yolov3-tiny.cfg"
cfg_custom = "cfg/yolov3-custom.cfg"
!cp cfg/{cfg_base} {cfg_custom}

!sed -i 's/batch=64/batch=32/' {cfg_custom}
!sed -i 's/subdivisions=16/subdivisions=8/' {cfg_custom}
!sed -i 's/max_batches = 500200/max_batches = 4000/' {cfg_custom}
!sed -i 's/steps=400000,450000/steps=3200,3600/' {cfg_custom}
!sed -i 's/classes=80/classes=2/' {cfg_custom}
!sed -i 's/filters=255/filters=21/' {cfg_custom}

# 🚀 Treinar modelo com mAP ativado
!./darknet detector train data/obj.data {cfg_custom} darknet53.conv.74 -map > treino.log

# 📈 Visualizar perda e mAP
import matplotlib.pyplot as plt

losses, maps = [], []
with open("treino.log", "r") as f:
    for line in f:
        if "avg loss" in line:
            parts = line.strip().split()
            loss = float(parts[parts.index("avg") + 2])
            losses.append(loss)
        if "mean average precision (mAP@0.50)" in line:
            m = float(line.strip().split()[-1])
            maps.append(m)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(losses, label="Loss")
plt.title("Perda durante o treinamento")
plt.xlabel("Iterações")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(maps, label="mAP", color="green")
plt.title("mAP durante o treinamento")
plt.xlabel("Validações")
plt.ylabel("mAP")
plt.legend()
plt.tight_layout()
plt.show()

# 📷 Upload de imagem para teste
from google.colab import files
uploaded = files.upload()
img_name = list(uploaded.keys())[0]

# 🧪 Rodar detecção
!./darknet detector test data/obj.data {cfg_custom} backup/yolov3-custom_best.weights {img_name} -dont_show

# 📸 Mostrar imagem original e com detecção
from PIL import Image
original = Image.open(img_name)
predicted = Image.open("predictions.png")

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(original)
plt.title("Imagem Original")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(predicted)
plt.title("Imagem com Detecção")
plt.axis("off")
plt.tight_layout()
plt.show()

# 📊 Avaliação com métricas
import numpy as np
import plotly.graph_objects as go
import cv2
from mean_average_precision import MetricBuilder

def convert_yolo_to_bbox(yolo_data, img_width, img_height, is_prediction=False):
    bboxes = []
    for line in yolo_data:
        parts = line.strip().split()
        cls = int(parts[0])
        x, y, w, h = map(float, parts[1:5])
        xmin = int((x - w / 2) * img_width)
        ymin = int((y - h / 2) * img_height)
        xmax = int((x + w / 2) * img_width)
        ymax = int((y + h / 2) * img_height)
        if is_prediction:
            conf = float(parts[5]) if len(parts) == 6 else 0.9
            bboxes.append([xmin, ymin, xmax, ymax, cls, conf])
        else:
            bboxes.append([xmin, ymin, xmax, ymax, cls])
    return bboxes

path_images = '../OIDv4_ToolKit/OID/Dataset/train/'
path_labels = '../OIDv4_ToolKit/OID/Dataset/train/'
path_preds = 'runs/detect/exp/labels/'  # Ajuste conforme sua saída

metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=False)

for filename in os.listdir(path_images):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(path_images, filename)
        label_path = os.path.join(path_labels, filename.replace('.jpg', '.txt').replace('.png', '.txt'))
        pred_path = os.path.join(path_preds, filename.replace('.jpg', '.txt').replace('.png', '.txt'))

        if not os.path.exists(label_path) or not os.path.exists(pred_path):
            continue

        img = cv2.imread(img_path)
        h, w = img.shape[:2]

        with open(label_path, 'r') as f:
            gt_data = f.readlines()
        with open(pred_path, 'r') as f:
            pred_data = f.readlines()

        gts = np.array(convert_yolo_to_bbox(gt_data, w, h, is_prediction=False))
        preds = np.array(convert_yolo_to_bbox(pred_data, w, h, is_prediction=True))

        metric_fn.add(preds, gts)

metrics = metric_fn.value()
precision = metrics['precision']
recall = metrics['recall']
f1_score = 2 * (precision * recall) / (precision + recall)

fig = go.Figure(data=[
    go.Bar(name='Precisão', x=['YOLO'], y=[precision], marker_color='skyblue'),
    go.Bar(name='Recall', x=['YOLO'], y=[recall], marker_color='orange'),
    go.Bar(name='F1-score', x=['YOLO'], y=[f1_score], marker_color='green')
])

fig.update_layout(
    title='📊 Avaliação do Modelo YOLO',
    yaxis=dict(title='Valor', range=[0, 1]),
    barmode='group',
    template='plotly_white'
)

fig.show()

# 📋 Relatório de qualidade do dataset + visualização
from collections import Counter

output_dir = "imagens_rotuladas"
os.makedirs(output_dir, exist_ok=True)

total_imgs = 0
sem_rotulo = 0
caixas_por_classe = Counter()

def salvar_com_bounding_boxes(img_path, label_path, output_path):
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    if not os.path.exists(label_path):
        return False, 0

    with open(label_path, "r") as f:
        lines = f.readlines()

    if len(lines) == 0:
        return False, 0

    for line in lines:
        cls, x, y, box_w, box_h = map(float, line.strip().split())
        xmin = int((x - box_w / 2) * w)
        ymin = int((y - box_h / 2) * h)
        xmax = int((x + box_w / 2) * w)
        ymax = int((y + box_h / 2) * h)

        color = (0, 255, 0) if int(cls) == 0 else (255, 0, 0)
        label = "Capacete" if int(cls) == 0 else "Colete"
        caixas_por_classe[label] += 1

        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color, 2)
        cv2.putText(img, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv2.imwrite(output_path, img)
    return True, len(lines)

# 🔄 Processar todas as imagens do dataset
img_files = [os.path.join(root, f) for root, _, files in os.walk(img_dir) for f in files if f.endswith(".jpg")]

for img_path in img_files:
    total_imgs += 1
    label_path = img_path.replace(".jpg", ".txt")
    output_path = os.path.join(output_dir, os.path.basename(img_path))
    sucesso, num_caixas = salvar_com_bounding_boxes(img_path, label_path, output_path)
    if not sucesso:
        sem_rotulo += 1

# 📋 Relatório de qualidade do dataset
print("📋 Relatório de Qualidade do Dataset")
print(f"Total de imagens processadas: {total_imgs}")
print(f"Imagens sem rótulo: {sem_rotulo}")
print(f"Imagens com rótulo: {total_imgs - sem_rotulo}")
print("Caixas detectadas por classe:")
for classe, count in caixas_por_classe.items():
    print(f" - {classe}: {count} caixas")
print(f"Imagens rotuladas salvas em: {output_dir}")

# 👀 Visualizar amostras rotuladas diretamente no notebook
from PIL import Image
import matplotlib.pyplot as plt

amostras = os.listdir(output_dir)[:5]  # seleciona 5 imagens rotuladas
plt.figure(figsize=(15, 5))
for i, nome in enumerate(amostras):
    img = Image.open(os.path.join(output_dir, nome))
    plt.subplot(1, 5, i+1)
    plt.imshow(img)
    plt.title(nome)
    plt.axis("off")
plt.tight_layout()
plt.show()
